In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# from langdetect import detect
# from langdetect.lang_detect_exception import LangDetectException
sns.set_style("whitegrid")

First we load the data, and convert the 'dates' column to a dates datatype.

In [2]:
df=pd.read_csv('data/Checkouts_2005_cleaned_TC.csv')
#There shouldn't be any NA because there isn't when I wrote to the file, but they are there...
df = df.dropna()
df['CheckoutDate'] = pd.to_datetime(df['CheckoutDate'])

df.head()

,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CheckoutDate,CleanedCreator
0,Physical,BOOK,3,The hole / Hiroko Oyamada ; translated from th...,"Oyamada, Hiroko, 1983-","Man woman relationships Fiction, Country life ...","New Directions Books,",2020,the hole,2023-04-01,hiroko oyamada
1,Digital,AUDIOBOOK,2,Anything But Fine (unabridged),Tobias Madden,"Romance, Young Adult Fiction, Young Adult Lite...","Recorded Books, LLC",2022,anything but fine,2023-04-01,madden tobias
2,Digital,EBOOK,2,True,Erin McCarthy,"Fiction, Literature, Romance","Penguin Group (USA), Inc.",2015,true,2023-04-01,erin mccarthy
3,Physical,BOOK,1,The ninth metal / Benjamin Percy.,"Percy, Benjamin","Comets Fiction, Mines and mineral resources Fi...","Mariner Books, Houghton Mifflin Harcourt,",2021,the ninth metal,2023-04-01,benjamin percy
4,Digital,EBOOK,2,Middlemarch (World Digital Library Edition),George Eliot,"Fiction, Literature",Barnes & Noble World Digital Library,2005,middlemarch,2023-04-01,eliot george


We want to make a new data frame with one row for each title in the library, and then add a column for each month with the number of checkouts of that title in that month. 

We note that based on the sorted Dataframe below the first month of data come
April 2005. Thus, we start our month column at April 2005. 

In [3]:
import datetime

months_of_interest=pd.date_range(start='2005-04-01', end='2024-08-01',freq='MS')


In [4]:
print(months_of_interest)

DatetimeIndex(['2005-04-01', '2005-05-01', '2005-06-01', '2005-07-01',
               '2005-08-01', '2005-09-01', '2005-10-01', '2005-11-01',
               '2005-12-01', '2006-01-01',
               ...
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01',
               '2024-07-01', '2024-08-01'],
              dtype='datetime64[ns]', length=233, freq='MS')


In [5]:
#Get the list of all the column names
month_columns = months_of_interest.strftime('%m/%d/%Y')
month_columns = month_columns.to_list()

df_columns = df.columns

total_columns = [*df_columns, *month_columns]
print(total_columns)

['UsageClass', 'MaterialType', 'Checkouts', 'Title', 'Creator', 'Subjects', 'Publisher', 'PublicationYear', 'CleanedTitle', 'CheckoutDate', 'CleanedCreator', '04/01/2005', '05/01/2005', '06/01/2005', '07/01/2005', '08/01/2005', '09/01/2005', '10/01/2005', '11/01/2005', '12/01/2005', '01/01/2006', '02/01/2006', '03/01/2006', '04/01/2006', '05/01/2006', '06/01/2006', '07/01/2006', '08/01/2006', '09/01/2006', '10/01/2006', '11/01/2006', '12/01/2006', '01/01/2007', '02/01/2007', '03/01/2007', '04/01/2007', '05/01/2007', '06/01/2007', '07/01/2007', '08/01/2007', '09/01/2007', '10/01/2007', '11/01/2007', '12/01/2007', '01/01/2008', '02/01/2008', '03/01/2008', '04/01/2008', '05/01/2008', '06/01/2008', '07/01/2008', '08/01/2008', '09/01/2008', '10/01/2008', '11/01/2008', '12/01/2008', '01/01/2009', '02/01/2009', '03/01/2009', '04/01/2009', '05/01/2009', '06/01/2009', '07/01/2009', '08/01/2009', '09/01/2009', '10/01/2009', '11/01/2009', '12/01/2009', '01/01/2010', '02/01/2010', '03/01/2010', '0

In [6]:
#Sample dataframe used for priliminary testing
df_sample = df.sample(10000)
#df_sample.head()

In [7]:
#Get the unique pairs from (CleanedTitle, CleanedCreator)
#pairs = df_sample[['CleanedTitle', 'CleanedCreator']].value_counts().reset_index()
pairs = df[['CleanedTitle', 'CleanedCreator']].value_counts().reset_index()
pairs = pairs.drop(columns='count')
pairs[month_columns] = 0

pairs.shape

/var/folders/b3/x1q3dl4x5bjfh37t7wlc0kmr0000gp/T/ipykernel_22992/3556791328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pairs[month_columns] = 0
/var/folders/b3/x1q3dl4x5bjfh37t7wlc0kmr0000gp/T/ipykernel_22992/3556791328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pairs[month_columns] = 0
/var/folders/b3/x1q3dl4x5bjfh37t7wlc0kmr0000gp/T/ipykernel_22992/3556791328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

(440731, 235)

In [8]:
# Create a dictionary to map (CleanedTitle, CleanedCreator) to the respective rows in 'pairs'
#This should be faster than searching via df['CleanedTitle] ==row.CleanedTitle
pairs_lookup = pairs.set_index(['CleanedTitle', 'CleanedCreator'])

# Convert 'CheckoutDate' to string in the desired format beforehand
#df_sample['CheckoutDate_str'] = df_sample['CheckoutDate'].dt.strftime('%m/%d/%Y')
df['CheckoutDate_str'] = df['CheckoutDate'].dt.strftime('%m/%d/%Y')


# Iterate over the rows in df using apply (faster than iterrows)
def update_months(row):
    date_str = row['CheckoutDate_str']
    if date_str in pairs.columns:
        # Use the (CleanedTitle, CleanedCreator) pair to update the correct row
        try:
            # Find the matching row in pairs by CleanedTitle and CleanedCreator
            pairs.at[pairs_lookup.index.get_loc((row['CleanedTitle'], row['CleanedCreator'])), date_str] += row['Checkouts']
        except KeyError:
            print('error')
            print(row['CleanedTitle'])
            print(row['CleanedCreator'])
            pass  # Skip if no matching (CleanedTitle, CleanedCreator) in pairs


# Apply the update_row function to each row in df
df.apply(update_months, axis=1)
#df_sample.apply(update_row, axis=1)

pairs.head()



,CleanedTitle,CleanedCreator,04/01/2005,05/01/2005,06/01/2005,07/01/2005,08/01/2005,09/01/2005,10/01/2005,11/01/2005,...,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024,05/01/2024,06/01/2024,07/01/2024,08/01/2024
0,fancy nancy,jane o'connor,0,0,0,0,0,0,0,0,...,116,78,103,93,92,104,0,0,39,78
1,pride and prejudice,austen jane,0,0,0,0,0,2,11,13,...,413,476,473,451,473,424,0,0,364,391
2,the wind in the willows,grahame kenneth,0,0,0,0,0,2,0,1,...,35,36,32,33,35,29,0,0,27,39
3,yu-gi-oh!,kazuki takahashi,0,0,0,0,0,0,0,0,...,5,3,1,2,1,1,0,0,4,2
4,the walking dead.,kirkman robert,0,0,0,0,0,0,0,0,...,5,5,6,12,15,1,0,0,4,2


In [10]:
pairs.shape

(440731, 235)

In [9]:
pairs.to_csv('data/cleaned_months.csv', index=False)

In [14]:
#Visual Checking
df_sample[df_sample.CleanedTitle =='watershed']


,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CheckoutDate,CleanedCreator,CheckoutDate_str
5298448,Physical,SOUNDDISC,7,Watershed [sound recording] / k.d. lang.,"lang, k. d. (Kathy Dawn), 1961-",Popular music 2001 2010,"Nonesuch,",2008,watershed,2011-10-01,k lang,10/01/2011
16401230,Physical,SOUNDDISC,2,Watershed / k.d. lang.,"lang, k. d. (Kathy Dawn), 1961-",Popular music 2001 2010,"Nonesuch,",2008,watershed,2018-12-01,k lang,12/01/2018
6965200,Physical,SOUNDDISC,5,Watershed [sound recording] / k.d. lang.,"lang, k. d. (Kathy Dawn), 1961-",Popular music 2001 2010,"Nonesuch,",2008,watershed,2013-04-01,k lang,04/01/2013
4157326,Physical,SOUNDDISC,13,Watershed [sound recording] / k.d. lang.,"lang, k. d. (Kathy Dawn), 1961-",Popular music 2001 2010,"Nonesuch,",2008,watershed,2010-06-01,k lang,06/01/2010


In [19]:
pairs['06/01/2010']

0        0
1       13
2        0
3        0
4        0
        ..
9727     0
9728     0
9729     0
9730     0
9731     0
Name: 06/01/2010, Length: 9732, dtype: int64